In [ ]:
import numpy as np
import pandas as pd
import mysql.connector
import requests
from bs4 import BeautifulSoup
from flashtext.keyword import KeywordProcessor
from nltk import tokenize
from nltk.corpus import stopwords

In [ ]:
#Managing given keywords
with open('Interest groups - 5th Sept.txt','r') as f:
    c=f.readlines()

In [ ]:
c[0]=c[0][3:]

In [ ]:
#len(c)


In [ ]:
keywords=np.array(c)

In [ ]:
for i in range(len(keywords)):
    if i == len(keywords)-1:
        keywords[i]=keywords[i]
    else:
        keywords[i]=keywords[i][:-1]

In [ ]:
#loads All sites file
with open('Quantcast-Top-Million.csv.txt','r') as f:
    sites=f.readlines()

In [ ]:
weblinks=[]
for i in range(6000,7100):
    j=sites[i][5:-1]
    if j=='Hidden profile':
        pass
    else:
        weblinks.append(j)

In [ ]:
for i in range(len(weblinks)):
    weblinks[i]='https://www.'+weblinks[i]

In [ ]:
#Sample
weblinks[:20]

In [ ]:
#Saving links in database

conn= mysql.connector.connect(host='localhost',user='sachin',passwd='mysqlsachin',database='interntask2')

mycursor=conn.cursor()

mycursor.execute("""DROP TABLE IF EXISTS weblink""")
mycursor.execute("CREATE TABLE weblinks(weblinks VARCHAR(800))")




query = "INSERT INTO randomlinks (weblinks) VALUES (%s)"
mycursor.executemany(query, [(list_item, ) for list_item in weblinks])

conn.commit()


In [ ]:
len(weblinks)

In [ ]:
kp0=KeywordProcessor()

for word in keywords:
    kp0.add_keyword(word)

In [ ]:
def gethtmlcontent(url):
    try:
        soup = BeautifulSoup(url, 'html.parser')  #Parse html code
        
        texts = soup.findAll(text=True)                 #find all text
        text_from_html=' '.join(texts)                   #join all text
    
    
    except Exception as error:
        print('Error occured: {}'.format(error))
        return []
    return text_from_html
    

In [ ]:
sw=stopwords.words('english')
to_remove=['[]','','1','()','||','=','.',',','\n',':',';','\\','//','/']

for i in to_remove:
    sw.append(i)

In [ ]:
def top_keywords(text):
    x=str(text)
    
    x=x.split(' ')
    words=[w for w in x]
    sws=set(sw)
    useful_words=[w for w in words if w not in sws]
    
    uniq=np.unique(useful_words,return_counts=True)
    
    if len(uniq[1])==0:
        return 0
    else:
        index=np.argmax(uniq[1])
    
    
    return uniq[0][index]
    

In [ ]:
def find_class(text_from_html):
    x=str(text_from_html)
    y0 = len(kp0.extract_keywords(x))
    if y0==0:
        top_kw=top_keywords(text_from_html)
        #Category='None'
        return 0,top_kw
    
    else:
        kw=kp0.extract_keywords(x)
        uniq_kw=np.unique(kw,return_counts=True)
        m=np.argmax(uniq_kw[1])
        max_kw=uniq_kw[1][m]
        #y0 = len(kp0.extract_keywords(x))




        return (max_kw/len(keywords))*100,kw


In [ ]:
confidences=[]
category=[]
links=[]
combine=[]
for i in range(len(weblinks)):
    url= weblinks[i]
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Cafari/537.36'}
    

    try:
        page = requests.get(url,headers=headers)       #to extract page from website
    except requests.exceptions.ConnectionError:
    
        i+=1
        continue
    sc=page.status_code
    if sc == 403:
        continue
    
    html_code = page.content       #to extract html code from page
    text_from_html=gethtmlcontent(html_code)
    
    confidence,kw=find_class(text_from_html)
    confidences.append(confidence)
    
    print("Confidience : ",confidence)
    uniqkw=np.unique(kw,return_counts=True)
    
    m=np.argmax(uniqkw[1])
    links.append(weblinks[i])
    
    print("Top_keyword: ",uniqkw[0][m])
    #print(i)
    print("Link: ",weblinks[i])
    category.append(uniqkw[0][m])
    combine.append((weblinks[i],confidence,uniqkw[0][m]))
    print('-'*25)
    

In [ ]:
combined=np.array(combine)

In [ ]:
conn= mysql.connector.connect(host='localhost',user='sachin',passwd='mysqlsachin',database='atginterntask2')

mycursor=conn.cursor()

mycursor.execute("""DROP TABLE IF EXISTS web_classification""")
mycursor.execute("CREATE TABLE web_classification(weblinks VARCHAR(800),confidence INT(200),kw VARCHAR(600))")


query = """INSERT INTO  web_classification(Links,Confidence,Top_Keyword) 
           VALUES (%s %s %s)"""

mycursor.executemany(query, combined.all)
                     
conn.commit()